In [1]:
import os

In [2]:
%pwd

'r:\\Projects\\Flight_Price_Prediction_ML\\Flight-Price-Prediction-ML-model\\research'

In [3]:
os.chdir('r:\\Projects\\Flight_Price_Prediction_ML\\Flight-Price-Prediction-ML-model')

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    train_data_path : Path
    test_data_path : Path
    model_name : str
    learning_rate: float
    max_depth:float
    n_estimators:float
    target_column : str


In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__ (
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH,
            label_encoding_filepath = LABEL_ENCODING_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)
            self.labels = read_yaml(label_encoding_filepath)

            create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
            config = self.config.model_trainer
            schema = self.schema.TARGET_COLUMN
            params = self.params.XGBRegressor

            create_directories([config.root_dir])

            model_trainer_config = ModelTrainerConfig(
                    root_dir= config.root_dir,
                    train_data_path=config.train_data_path,
                    test_data_path= config.test_data_path,
                    model_name=config.model_name,
                    learning_rate=params.learning_rate,
                    max_depth=params.max_depth,
                    n_estimators=params.n_estimators,
                    target_column=schema.name

            )

            return model_trainer_config

In [7]:
import pandas as pd
import os
from mlProject import logger
from xgboost import XGBRegressor

import joblib


class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x =  test_data.drop([self.config.target_column],axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        Xr = XGBRegressor(learning_rate=self.config.learning_rate, max_depth=self.config.max_depth, n_estimators = self.config.n_estimators, random_state=42)
        Xr.fit(train_x,train_y)

        joblib.dump(Xr,os.path.join(self.config.root_dir,self.config.model_name))




In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()

except Exception as e:
    raise e

[2024-07-14 02:12:59,883: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-14 02:12:59,900: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-14 02:12:59,916: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-14 02:12:59,935: INFO: common: yaml file: label_encoding.yaml loaded successfully]
[2024-07-14 02:12:59,940: INFO: common: created directory at: artifacts]
[2024-07-14 02:12:59,944: INFO: common: created directory at: artifacts/model_trainer]
